# Files Management

WAL files management is a crucial procedure that is mostly automated but sometimes need to be configured and interfered with.

## Switch WAL File

We already saw how to manually switch WAL file, there are flows that cause the switching:

- WAL segment is filled up.
- The function `pg_wal_switch` is called.
- The `archive_mode` parameter is enabled and the `archive_timeout` parameter has expired.

## Creation, Deletion And Recycling Of Files

### Recycle & Deletion

On checkpoint:
- Estimate the number of needed WAL files for the next CHECKPOINT period by checking the last needed file\
(also `min_wal_size` < files_num <  `max_wal_size`)
- Recycle needed amount of WAL files
- Remove what left (up to `wal_keep_size`)

Illustration:

<img src="./helpers/recycle-wal-files-checkpoint.png " alt="drawing" width="700"/>

### Creation

Created when the existing WAL files are filled up

## Archiving - Enables PITR Backup Strategy

When Archiving is enabled (`archive_mode` - disabled by default) every time a WAL file is switched it declared as "done" and can be archived.

The archiving process can be achieved with two approaches:
- Shell
- Library

### Shell

#### Usage

Using shell commands that are invoked every time a WAL file is switched, for example:

`archive_command` = '`'test ! -f /mnt/server/archivedir/%f && cp %p /mnt/server/archivedir/%f'`'

- %p: Relative WAL file path + name
- %f: WAL file name

#### Exit Code

It is important that the archive command return zero exit status if and only if it succeeds! Upon getting a zero result, PostgreSQL will assume that the file has been successfully archived, and will remove or recycle it. However, a nonzero status tells PostgreSQL that the file was not archived; it will try again periodically until it succeeds.

#### Don't Overwrite

Archiving should be designed to refuse overwriting pre-existing files to avoid corrupting an existing WAL archive of a different server

In rare cases the same WAL file can be a subject to archiving twice due to a server crash, if the new and pre-existing WAL files are identical in content, you can return a zero exit status code

#### When Problems Occur

- Make sure errors are propagated somehow to the DBA
- Delays in archiving are often not a problem, but make sure the wal directory doesn't bloat
- If an archiving strategy is not working, you can work on a solution and meanwhile change a strategy, most probably using shell instead of a library for a small period of time. This can be achieved without downtime with configuration change and server reload.

### Library

Implement more complex logic for archiving with custom code and implementation of the necessary callbacks

Use: 

####  Cleanup

The cleanup of an archive is not done automatically you should cleanup by yourself using the shell or `pg_archivecleanup`

It can be scheduled by: `archive_cleanup_command` that invokes the specified command on restart points

Archive libraries can implement their own cleanup strategies using callbacks and background processes

### Live Demo

1. Find configuration file `postgresql.conf`
1. Create custom directory for archives (change owner)
1. Change `archive_mode` and `archive_command`
1. Start (if it was shutdown) / Restart server
1. Insert some rows to enforce new WAL file
1. Check the archive directory to see if it's done
1. Check out the archive statistics on `pg_catalog.pg_stat_archiver`